In [208]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns

In [209]:
# Load the penguins dataset
penguins = sns.load_dataset('penguins')
# Drop rows with missing values
penguins.dropna(inplace=True)
# Features
#X = penguins[['bill_length_mm', 'bill_depth_mm', 'flipper_length_mm', 'body_mass_g']]
X = penguins[['bill_length_mm', 'bill_depth_mm']]

# Target: 1 if Adelie, 0 otherwise
y = (penguins['species'] == 'Adelie').astype(int)

In [210]:
#scaler = StandardScaler()
#scaler= MinMaxScaler()
#X = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)
X= X / X.max()

In [211]:
model = LogisticRegression(C=1, solver="saga", fit_intercept=False, max_iter=1000)
model.fit(X, y)

LogisticRegression(C=1, fit_intercept=False, max_iter=1000, solver='saga')

In [212]:
# Print model coefficients
print("Model Coefficients:")
for feature, coef in zip(X.columns, model.coef_[0]):
    print(f"{feature}: {coef:.10f}")
print(f"Intercept: {model.intercept_[0]:.410}\n")

Model Coefficients:
bill_length_mm: -6.3259882493
bill_depth_mm: 5.4492302332
Intercept: 0.0



In [213]:
def manual_prob(row):
    linear_sum = intercept + np.dot(coefficients, row)
    return 1 / (1 + np.exp(-linear_sum))

In [214]:
y_prob_manual = X.apply(manual_prob, axis=1)

In [215]:
# Create results DataFrame
results = X.copy()
results['Actual'] = y.values
results['Predicted'] = model.predict(X)
results['Probability_Adelie'] = model.predict_proba(X)[:,1]
results['Probability_Adelie_Manual'] = y_prob_manual

# Show first few rows
print(results.head(10))

    bill_length_mm  bill_depth_mm  Actual  Predicted  Probability_Adelie  \
0         0.656040       0.869767       1          1            0.643241   
1         0.662752       0.809302       1          1            0.554165   
2         0.676174       0.837209       1          1            0.570692   
4         0.615772       0.897674       1          1            0.730324   
5         0.659396       0.958140       1          1            0.740735   
6         0.652685       0.827907       1          1            0.594495   
7         0.657718       0.911628       1          1            0.691460   
12        0.689597       0.818605       1          1            0.524575   
13        0.647651       0.986047       1          1            0.781796   
14        0.580537       0.981395       1          1            0.842292   

    Probability_Adelie_Manual  
0                    0.643235  
1                    0.554162  
2                    0.570688  
4                    0.730316  
5  

In [216]:
model.score(X, y)

0.9819819819819819

C:\Users\jens.nilsen\python\WPy64-31230\python-3.12.3.amd64\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(


In [217]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

x_min, x_max = 0.5, 1
y_min, y_max = 0.5, 1

# Coefficients from scikit-learn LogisticRegression
coefficients = model.coef_.flatten()  # shape (2,)
slope = (-coefficients[0] / coefficients[1]).item()  # scalar
intercept = 0  # no intercept

# Line for plotting
x_vals = np.array([float(x_min), float(x_max)])
y_vals = slope * x_vals + intercept

# Create meshgrid
xx, yy = np.meshgrid(np.linspace(x_min, x_max, 300),
                     np.linspace(y_min, y_max, 300))
grid = np.c_[xx.ravel(), yy.ravel()]
Z = model.predict_proba(grid)[:, 1].reshape(xx.shape)

# Plot
plt.figure(figsize=(8,6))
plt.contourf(xx, yy, Z, levels=[0,0.5,1], cmap='RdBu', alpha=0.6)
plt.contour(xx, yy, Z, levels=[0.5], colors='black', linewidths=2)

# Scatter plot
y_series = pd.Series(y, index=X.index)  # do not overwrite y
plt.scatter(X.loc[y_series==1, 'bill_length_mm'], X.loc[y_series==1, 'bill_depth_mm'],
            c='blue', edgecolor='k', label='Adelie')
plt.scatter(X.loc[y_series==0, 'bill_length_mm'], X.loc[y_series==0, 'bill_depth_mm'],
            c='red', edgecolor='k', label='Other')

plt.xlim(x_min, x_max)
plt.ylim(y_min, y_max)

# Decision line
plt.plot(x_vals, y_vals, 'k-', label=f"Decision line: depth = {slope:.4f} * length")


plt.xlabel('Bill length (standardized)')
plt.ylabel('Bill depth (standardized)')
plt.title('Decision Boundary (Adelie vs Others)')
plt.legend()
plt.show()
